# カテゴリ値を含むデータでのLightGBMを使った分析

### データ
[Adult Data Set](https://archive.ics.uci.edu/ml/datasets/adult) \
[Adult Census Income](https://www.kaggle.com/datasets/uciml/adult-census-income) \
[Adult income dataset](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset)

### 参考
[lightgbmで二値分類の一連の流れをしたメモ](https://qiita.com/d_desuyon/items/807e01311ad08570ee78) \
[【初心者向け】特徴量重要度の算出 (LightGBM) 【Python】【機械学習】](https://mathmatical22.xyz/2020/04/12/%E3%80%90%E5%88%9D%E5%BF%83%E8%80%85%E5%90%91%E3%81%91%E3%80%91%E7%89%B9%E5%BE%B4%E9%87%8F%E9%87%8D%E8%A6%81%E5%BA%A6%E3%81%AE%E7%AE%97%E5%87%BA-lightgbm-%E3%80%90python%E3%80%91%E3%80%90%E6%A9%9F/) \
[LightGBMのCategorical Featureによって精度が向上するか？](https://qiita.com/sinchir0/items/b038757e578b790ec96a) \
[Pythonでしか描けない美しいデシジョンツリー（決定木）をlightGBMでも描こう！](https://qiita.com/hima2b4/items/b356e688ebdabc393da8) \
[LightGBM/XGBoostのtree構造可視化メモ](https://qiita.com/dyamaguc/items/70d4ce3548d49a6256fa) \
[Category Encodersでカテゴリ特徴量をストレスなく変換する](https://qiita.com/Hyperion13fleet/items/afa49a84bd5db65ffc31) \
[カテゴリ変数系特徴量の前処理(scikit-learnとcategory_encoders)](https://qiita.com/FukuharaYohei/items/06cc7b1c5eb124db6cdf) \
[LightGBMのCategorical Featureによって精度が向上するか？](https://qiita.com/sinchir0/items/b038757e578b790ec96a) \
[LightGBMでOptunaを使用するときの再現性確保について](https://book-read-yoshi.hatenablog.com/entry/2021/03/22/lightgbm_optuna_deterministic)

In [1]:
!python -V

Python 3.9.13


In [2]:
!python -m pip --version

pip 23.0.1 from C:\Python39\lib\site-packages\pip (python 3.9)



In [3]:
!python -m pip list

Package                  Version
------------------------ -----------
aiofiles                 22.1.0
aiosqlite                0.18.0
alembic                  1.9.4
anyio                    3.6.2
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.2.1
attrs                    22.2.0
Babel                    2.11.0
backcall                 0.2.0
beautifulsoup4           4.11.2
bleach                   6.0.0
bs4                      0.0.1
category-encoders        2.6.0
certifi                  2022.12.7
cffi                     1.15.1
charset-normalizer       3.0.1
cmaes                    0.9.1
colorama                 0.4.6
colorlog                 6.7.0
comm                     0.1.2
contourpy                1.0.7
cycler                   0.11.0
debugpy                  1.6.6
decorator                5.1.1
defusedxml               0.7.1
executing                1.2.0
fastjsonschema           2.16.2
fonttools        

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 100)
import numpy as np
from sklearn.model_selection import train_test_split
#import lightgbm as lgb
import optuna
# ログを非表示にしたいときは、optuna.logging.disable_default_handler()
# 逆に、表示にしたいときは、optuna.logging.enable_default_handler()
optuna.logging.disable_default_handler()
# optuna経由でLightGBMをインポート
from optuna.integration import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, auc, roc_curve, precision_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('adult_uci_label.csv')

In [6]:
df.head(10)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,82,3,132870,11,9,6,3,1,4,0,0,4356,18,38,0
1,54,3,140359,5,4,0,6,4,4,0,0,3900,40,38,0
2,41,3,264663,15,10,5,9,3,4,0,0,3900,40,38,0
3,34,3,216864,11,9,0,7,4,4,0,0,3770,45,38,0
4,38,3,150601,0,6,5,0,4,4,1,0,3770,40,38,0
5,74,6,88638,10,16,4,9,2,4,0,0,3683,20,38,1
6,68,0,422013,11,9,0,9,1,4,0,0,3683,40,38,0
7,45,3,172274,10,16,0,9,4,2,0,0,3004,35,38,1
8,38,5,164526,14,15,4,9,1,4,1,0,2824,45,38,1
9,52,3,129177,9,13,6,7,1,4,0,0,2824,20,38,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             30162 non-null  int64
 1   workclass       30162 non-null  int64
 2   fnlwgt          30162 non-null  int64
 3   education       30162 non-null  int64
 4   education.num   30162 non-null  int64
 5   marital.status  30162 non-null  int64
 6   occupation      30162 non-null  int64
 7   relationship    30162 non-null  int64
 8   race            30162 non-null  int64
 9   sex             30162 non-null  int64
 10  capital.gain    30162 non-null  int64
 11  capital.loss    30162 non-null  int64
 12  hours.per.week  30162 non-null  int64
 13  native.country  30162 non-null  int64
 14  income          30162 non-null  int64
dtypes: int64(15)
memory usage: 3.5 MB


In [8]:
columns_cat = [
    'workclass',
    'education',
    'marital.status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native.country',
    #'income',
]

In [9]:
{cat:'category' for cat in columns_cat}

{'workclass': 'category',
 'education': 'category',
 'marital.status': 'category',
 'occupation': 'category',
 'relationship': 'category',
 'race': 'category',
 'sex': 'category',
 'native.country': 'category'}

In [10]:
df = df.astype({cat:'category' for cat in columns_cat})

In [11]:
for cat in columns_cat:
    print(cat, df[cat].cat.categories)

workclass Int64Index([0, 1, 3, 4, 5, 6, 7], dtype='int64')
education Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')
marital.status Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')
occupation Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')
relationship Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')
race Int64Index([0, 1, 2, 3, 4], dtype='int64')
sex Int64Index([0, 1], dtype='int64')
native.country Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40],
           dtype='int64')


In [12]:
df['income'].value_counts()

0    22654
1     7508
Name: income, dtype: int64

In [13]:
df.head(10)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,82,3,132870,11,9,6,3,1,4,0,0,4356,18,38,0
1,54,3,140359,5,4,0,6,4,4,0,0,3900,40,38,0
2,41,3,264663,15,10,5,9,3,4,0,0,3900,40,38,0
3,34,3,216864,11,9,0,7,4,4,0,0,3770,45,38,0
4,38,3,150601,0,6,5,0,4,4,1,0,3770,40,38,0
5,74,6,88638,10,16,4,9,2,4,0,0,3683,20,38,1
6,68,0,422013,11,9,0,9,1,4,0,0,3683,40,38,0
7,45,3,172274,10,16,0,9,4,2,0,0,3004,35,38,1
8,38,5,164526,14,15,4,9,1,4,1,0,2824,45,38,1
9,52,3,129177,9,13,6,7,1,4,0,0,2824,20,38,1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             30162 non-null  int64   
 1   workclass       30162 non-null  category
 2   fnlwgt          30162 non-null  int64   
 3   education       30162 non-null  category
 4   education.num   30162 non-null  int64   
 5   marital.status  30162 non-null  category
 6   occupation      30162 non-null  category
 7   relationship    30162 non-null  category
 8   race            30162 non-null  category
 9   sex             30162 non-null  category
 10  capital.gain    30162 non-null  int64   
 11  capital.loss    30162 non-null  int64   
 12  hours.per.week  30162 non-null  int64   
 13  native.country  30162 non-null  category
 14  income          30162 non-null  int64   
dtypes: category(8), int64(7)
memory usage: 1.8 MB


In [15]:
# 不要カラムの削除と並べ替え
df = df[["age",
         "workclass",
         #"fnlwgt",
         "education",
         "education.num",
         "marital.status",
         "occupation",
         "relationship",
         "race",
         "sex",
         "capital.gain",
         "capital.loss",
         "hours.per.week",
         "native.country",
         "income",]]
df.head(10)

,age,workclass,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,82,3,11,9,6,3,1,4,0,0,4356,18,38,0
1,54,3,5,4,0,6,4,4,0,0,3900,40,38,0
2,41,3,15,10,5,9,3,4,0,0,3900,40,38,0
3,34,3,11,9,0,7,4,4,0,0,3770,45,38,0
4,38,3,0,6,5,0,4,4,1,0,3770,40,38,0
5,74,6,10,16,4,9,2,4,0,0,3683,20,38,1
6,68,0,11,9,0,9,1,4,0,0,3683,40,38,0
7,45,3,10,16,0,9,4,2,0,0,3004,35,38,1
8,38,5,14,15,4,9,1,4,1,0,2824,45,38,1
9,52,3,9,13,6,7,1,4,0,0,2824,20,38,1


In [16]:
X = df.drop(['income'], axis=1)
y = df['income']

#Creation of Train and Test dataset
X_train, X_test , y_train, y_test  = train_test_split(X      , y      , test_size=0.2, shuffle=False)

#Creation of Train and validation dataset
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

In [17]:
print ("Train dataset: {0}{1}".format(X_train.shape, y_train.shape))
print ("Validation dataset: {0}{1}".format(X_valid.shape, y_valid.shape))
print ("Test dataset: {0}{1}".format(X_test.shape, y_test.shape))

Train dataset: (19303, 13)(19303,)
Validation dataset: (4826, 13)(4826,)
Test dataset: (6033, 13)(6033,)


In [18]:
# データセットを生成する
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [19]:
# LightGBMのハイパーパラメータ
params = {
    # 勾配ブースティング木(GBDT:Gradient Boosting Decision Tree)
    'boosting_type': 'gbdt',
    # 二値分類問題
    'objective': 'binary',
    # AUCの最大化を目指す
    'metric': 'auc',
    # 正答率の最大化を目指す
    #'metric': 'binary_error',
    # Fatal の場合出力
    'verbosity': -1,
    # 学習率
    'learning_rate': 0.02,
    'deterministic': True, # 再現性確保用のパラメータ
    'force_row_wise': True  # 再現性確保用のパラメータ
}

In [20]:
# 乱数Seedを指定した試行
# LightGBMTunerのインスタンス化、学習（≒探索）、最適モデル（およびパラメータ）取得などの流れは前節に同じ
# 更にモデル精度指標算出を追加
def autotuning(seed=31):
    booster = lgb.LightGBMTuner(
        params = params,
        train_set = lgb_train,
        valid_sets = lgb_valid,
        verbose_eval = 50,  # 50イテレーション毎に学習結果出力
        num_boost_round = 10000,  # 最大イテレーション回数指定
        early_stopping_rounds = 100,
        categorical_feature = columns_cat, # categorical_featureを設定
        optuna_seed = seed, # 再現性確保用のパラメータ
    )

    # 上記のパラメータでモデルを学習する
    booster.run()

    # 最適なパラメータの表示
    #print("Best params:", booster.best_params)

    # 最適なモデル（Boosterオブジェクト）を取得する
    model = booster.get_best_booster()

    # 最適なパラメータの表示
    best_params = model.params
    print("Best params:", best_params)

    # テストデータを予測する
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # AUCを計算
    fpr, tpr, thresholds = roc_curve(np.asarray(y_test), y_pred)
    #print("AUC", auc(fpr, tpr))

    metrics = {
        "seed": seed,
        "auc": auc(fpr, tpr),
        "accuracy": accuracy_score(np.asarray(y_test), np.round(y_pred)),
        "precision": precision_score(np.asarray(y_test), np.round(y_pred)),
    }

    # Tupleとして戻り値に
    return (best_params, metrics)

In [21]:
# 乱数シードを指定（from 素数を取得するワンライナー）
seeds = (lambda n:[x for x in range(2, n) if not 0 in map(lambda z: x%z, range(2, x))])(10)

In [22]:
import csv

# 各試行での情報を保存
listParams = []
listMetrics = []

# 乱数Seedを指定した試行を自動化
for s in seeds:
    print("### seed:%3s ###" %s)
    tunedparam, metrics = autotuning(s)
    tunedparam["seed"] = s
    listParams.append(tunedparam)
    listMetrics.append(metrics)

# DataFrameからCSVファイルを生成
# encoding="SJIS"だとJupyterLab（CSVTable）上で表示不可なことに注意
df_params = pd.DataFrame(listParams)
df_params.to_csv('lightgbm_label_params.csv', index=False, header=True, encoding='UTF-8', sep=',', quotechar='"', quoting=csv.QUOTE_ALL)
df_metrics = pd.DataFrame(listMetrics)
df_metrics.to_csv('lightgbm_label_metrics.csv', index=False, header=True, encoding='UTF-8', sep=',', quotechar='"', quoting=csv.QUOTE_ALL)

### seed:  2 ###


feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds


C:\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Python39\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['education', 'marital.status', 'native.country', 'occupation', 'race', 'relationship', 'sex', 'workclass']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C

[50]	valid_0's auc: 0.902664
[100]	valid_0's auc: 0.904155
[150]	valid_0's auc: 0.90514
[200]	valid_0's auc: 0.905737
[250]	valid_0's auc: 0.906364
[300]	valid_0's auc: 0.90656
[350]	valid_0's auc: 0.906693
[400]	valid_0's auc: 0.906715
[450]	valid_0's auc: 0.906831


...

[550]	valid_0's auc: 0.90705
[600]	valid_0's auc: 0.906953
Early stopping, best iteration is:
[509]	valid_0's auc: 0.907135
Training until validation scores don't improve for 100 rounds


C:\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Python39\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['education', 'marital.status', 'native.country', 'occupation', 'race', 'relationship', 'sex', 'workclass']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C

[50]	valid_0's auc: 0.90168
[100]	valid_0's auc: 0.903043
[150]	valid_0's auc: 0.904106
[200]	valid_0's auc: 0.905042
[250]	valid_0's auc: 0.905927
[300]	valid_0's auc: 0.906486
[350]	valid_0's auc: 0.906661
[400]	valid_0's auc: 0.906859
[450]	valid_0's auc: 0.907


min_data_in_leaf, val_score: 0.907135: 100%|#############################################| 5/5 [00:05<00:00,  1.04s/it]

[500]	valid_0's auc: 0.907114
[550]	valid_0's auc: 0.907078
Early stopping, best iteration is:
[492]	valid_0's auc: 0.907128
Best params: {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'learning_rate': 0.02, 'deterministic': True, 'force_row_wise': True, 'feature_pre_filter': False, 'lambda_l1': 1.7707045283284145e-06, 'lambda_l2': 4.225646242357928, 'num_leaves': 21, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 100, 'num_iterations': 10000, 'early_stopping_round': 100, 'categorical_column': [1, 2, 4, 5, 6, 7, 8, 12]}


In [23]:
# 乱数シードで照準にソート
df_params.sort_values("seed", ascending=True)

,boosting_type,objective,metric,verbosity,learning_rate,deterministic,force_row_wise,feature_pre_filter,lambda_l1,lambda_l2,num_leaves,feature_fraction,bagging_fraction,bagging_freq,min_child_samples,num_iterations,early_stopping_round,categorical_column,seed
0,gbdt,binary,auc,-1,0.02,True,True,False,4.976134e-06,0.000003,31,0.4,0.661597,1,20,10000,100,"[1, 2, 4, 5, 6, 7, 8, 12]",2
1,gbdt,binary,auc,-1,0.02,True,True,False,2.129894e-06,7.313785,31,0.4,0.935768,7,20,10000,100,"[1, 2, 4, 5, 6, 7, 8, 12]",3
2,gbdt,binary,auc,-1,0.02,True,True,False,9.952646e-07,0.686421,20,0.4,0.859545,4,25,10000,100,"[1, 2, 4, 5, 6, 7, 8, 12]",5
3,gbdt,binary,auc,-1,0.02,True,True,False,1.770705e-06,4.225646,21,0.4,1.000000,0,100,10000,100,"[1, 2, 4, 5, 6, 7, 8, 12]",7


In [24]:
# AUCで降順にソート
df_metrics.sort_values("auc", ascending=False)

,seed,auc,accuracy,precision
3,7,0.894849,0.855462,0.685330
1,3,0.894446,0.854964,0.679825
2,5,0.894323,0.856788,0.688543
0,2,0.893851,0.855130,0.679781
